In [8]:
import pprint
import requests
import csv
import datetime
import pandas as pd
import time
import nltk
import re
from pandas.io.json import json_normalize
from sklearn.cluster import SpectralClustering
from math import sqrt, floor

pd.set_option("display.max_columns", 50)

In [9]:
results = requests.get("https://api.nytimes.com/svc/books/v3/lists/2023-05-25/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb")

In [10]:
results.status_code

200

In [11]:
BOOKS_ROOT = "https://api.nytimes.com/svc/books/v3/lists"
LIST = "audio-fiction"
API_KEY = 'keys.json'

def subtract_months(date, months):
    year = date.year
    month = date.month
    new_month = month - months
    if new_month <= 0:
        year -= (abs(new_month) // 12) + 1
        new_month = 12 - (abs(new_month) % 12)
    return date.replace(year=year, month=new_month)

bestsellers = []
months = []
num_calls_to_make = 1
months_per_call = 62  # Number of months to fetch per API call

for call_num in range(num_calls_to_make):
    ctr = call_num * months_per_call
    start_date = datetime.date(2023, 5, 14)
    start_date = subtract_months(start_date, ctr)
    end_date = subtract_months(start_date, months_per_call)
    
    for i in range(months_per_call):
        curr_date = subtract_months(start_date, i)
        print(curr_date)
        url = f"{BOOKS_ROOT}/{curr_date}/{LIST}.json?api-key={API_KEY}"
        url = url.strip()
        print(url)
        r = requests.get(url)
        
        if r.status_code == 200:
            results = r.json()
            for book in results['results']['books']:
                book_info = {
                    'title': book['title'],
                    'description': book['description'],
                    'contributor': book['contributor'],
                    'author': book['author'],
                    'contributor_note': book['contributor_note'],
                    'price': book['price'],
                    'age_group': book['age_group'],
                    'publisher': book['publisher'],
                    'primary_isbn13': book['primary_isbn13'],
                    'primary_isbn10': book['primary_isbn10'],
                    'rank': book['rank'],
                    'rank_last_week': book['rank_last_week'],
                    'weeks_on_list': book['weeks_on_list'],
                    'asterisk': book['asterisk'],
                    'dagger': book['dagger']
                }
                bestsellers.append(book_info)
            months.append(str(curr_date))
        else:
            print("API call did not work!")
            print(r)
        
        time.sleep(15)

2023-05-14
https://api.nytimes.com/svc/books/v3/lists/2023-05-14/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-04-14
https://api.nytimes.com/svc/books/v3/lists/2023-04-14/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-03-14
https://api.nytimes.com/svc/books/v3/lists/2023-03-14/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-02-14
https://api.nytimes.com/svc/books/v3/lists/2023-02-14/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2023-01-14
https://api.nytimes.com/svc/books/v3/lists/2023-01-14/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2022-12-14
https://api.nytimes.com/svc/books/v3/lists/2022-12-14/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2022-11-14
https://api.nytimes.com/svc/books/v3/lists/2022-11-14/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb
2022-10-14
https://api.nytimes.com/svc/books/v3/lists/2022-10-14/audio-fiction.json?api-key=P1yDSg62Aydsdh04WBBuS57pYbpBu2tb


In [12]:
audio_fiction = pd.json_normalize(bestsellers)

In [13]:
audio_fiction.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
0,THE LAST THING HE TOLD ME,Hannah Hall discovers truths about her missing...,by Laura Dave,Laura Dave,,0.00,,Simon & Schuster Audio,9781797123196,,1,0,0,0,0
1,LESSONS IN CHEMISTRY,A scientist and single mother living in Califo...,by Bonnie Garmus,Bonnie Garmus,,0.00,,Random House Audio,9780593507537,0593507533,2,0,0,0,0
2,REMARKABLY BRIGHT CREATURES,A widow working the night shift at the Sowell ...,by Shelby Van Pelt,Shel Van Pelt,,0.00,,HarperAudio,9780063204188,0063204185,3,0,0,0,0
3,HELLO BEAUTIFUL,In a homage to Louisa May Alcott’s “Little Wom...,by Ann Napolitano,Ann Napolitano,,0.00,,Random House Audio,9780593664049,0593664043,4,0,0,0,0
4,SIMPLY LIES,A former detective becomes the prime suspect i...,by David Baldacci,David Baldacci,,0.00,,Hachette Audio,9781549152016,,5,0,0,0,0


In [14]:
len(audio_fiction)

930

In [15]:
audio_fiction.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger
925,THE SECRETS SHE KEEPS,Two pregnant women from different walks of lif...,by Michael Robotham,Michael Robotham,,0.00,,Simon & Schuster Audio,9781508237396,,11,0,0,0,0
926,THE ROOSTER BAR,Three students at a sleazy for-profit law scho...,by John Grisham,John Grisham,,0.00,,Random House Audio,9780399565007,,12,0,0,0,0
927,THE WIFE,A woman with a tragic past must choose between...,by Alafair Burke,Alafair Burke,,0.00,,HarperAudio,9780062798473,0062798472,13,0,0,0,0
928,THE GRAY MAN,"Court Gentry, a former C.I.A. operative, is ch...",by Mark Greaney,Mark Greaney,,0.00,,Audible Studios,9781603933261,,14,0,0,0,0
929,ORIGIN,A symbology professor goes on a perilous quest...,by Dan Brown,Dan Brown,,0.00,,Random House Audio,9780739319307,0739319302,15,0,0,0,0


In [16]:
len(months)

62

In [17]:
#repeat months 15 times to add to df
#https://www.pythonforbeginners.com/lists/repeat-each-element-in-a-list-in-python
k = 15
repeated_months = []
for element in months:
    for i in range(k):
       repeated_months.append(element)

In [18]:
len(repeated_months)

930

In [19]:
audio_fiction['month'] = repeated_months

In [20]:
audio_fiction.head()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,month
0,THE LAST THING HE TOLD ME,Hannah Hall discovers truths about her missing...,by Laura Dave,Laura Dave,,0.00,,Simon & Schuster Audio,9781797123196,,1,0,0,0,0,2023-05-14
1,LESSONS IN CHEMISTRY,A scientist and single mother living in Califo...,by Bonnie Garmus,Bonnie Garmus,,0.00,,Random House Audio,9780593507537,0593507533,2,0,0,0,0,2023-05-14
2,REMARKABLY BRIGHT CREATURES,A widow working the night shift at the Sowell ...,by Shelby Van Pelt,Shel Van Pelt,,0.00,,HarperAudio,9780063204188,0063204185,3,0,0,0,0,2023-05-14
3,HELLO BEAUTIFUL,In a homage to Louisa May Alcott’s “Little Wom...,by Ann Napolitano,Ann Napolitano,,0.00,,Random House Audio,9780593664049,0593664043,4,0,0,0,0,2023-05-14
4,SIMPLY LIES,A former detective becomes the prime suspect i...,by David Baldacci,David Baldacci,,0.00,,Hachette Audio,9781549152016,,5,0,0,0,0,2023-05-14


In [21]:
audio_fiction.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,month
925,THE SECRETS SHE KEEPS,Two pregnant women from different walks of lif...,by Michael Robotham,Michael Robotham,,0.00,,Simon & Schuster Audio,9781508237396,,11,0,0,0,0,2018-04-14
926,THE ROOSTER BAR,Three students at a sleazy for-profit law scho...,by John Grisham,John Grisham,,0.00,,Random House Audio,9780399565007,,12,0,0,0,0,2018-04-14
927,THE WIFE,A woman with a tragic past must choose between...,by Alafair Burke,Alafair Burke,,0.00,,HarperAudio,9780062798473,0062798472,13,0,0,0,0,2018-04-14
928,THE GRAY MAN,"Court Gentry, a former C.I.A. operative, is ch...",by Mark Greaney,Mark Greaney,,0.00,,Audible Studios,9781603933261,,14,0,0,0,0,2018-04-14
929,ORIGIN,A symbology professor goes on a perilous quest...,by Dan Brown,Dan Brown,,0.00,,Random House Audio,9780739319307,0739319302,15,0,0,0,0,2018-04-14


In [22]:
#and a column indicating which list these come from
k2 = 930
monthly_fiction_list = ['Audio Fiction']
nyt_list_monthly_fiction = []

for element in monthly_fiction_list:
    for i in range(k2):
        nyt_list_monthly_fiction.append(element)

len(nyt_list_monthly_fiction)

930

In [23]:
audio_fiction['nyt_list'] = nyt_list_monthly_fiction

In [24]:
audio_fiction.tail()

,title,description,contributor,author,contributor_note,price,age_group,publisher,primary_isbn13,primary_isbn10,rank,rank_last_week,weeks_on_list,asterisk,dagger,month,nyt_list
925,THE SECRETS SHE KEEPS,Two pregnant women from different walks of lif...,by Michael Robotham,Michael Robotham,,0.00,,Simon & Schuster Audio,9781508237396,,11,0,0,0,0,2018-04-14,Audio Fiction
926,THE ROOSTER BAR,Three students at a sleazy for-profit law scho...,by John Grisham,John Grisham,,0.00,,Random House Audio,9780399565007,,12,0,0,0,0,2018-04-14,Audio Fiction
927,THE WIFE,A woman with a tragic past must choose between...,by Alafair Burke,Alafair Burke,,0.00,,HarperAudio,9780062798473,0062798472,13,0,0,0,0,2018-04-14,Audio Fiction
928,THE GRAY MAN,"Court Gentry, a former C.I.A. operative, is ch...",by Mark Greaney,Mark Greaney,,0.00,,Audible Studios,9781603933261,,14,0,0,0,0,2018-04-14,Audio Fiction
929,ORIGIN,A symbology professor goes on a perilous quest...,by Dan Brown,Dan Brown,,0.00,,Random House Audio,9780739319307,0739319302,15,0,0,0,0,2018-04-14,Audio Fiction


In [25]:
audio_fiction.to_csv("C:\\Users\\melan\\Documents\\DA8\\Projects\\capstone\\data\\2018to2023_audio_fiction.csv", index=False)